<a href="https://colab.research.google.com/github/devShaurya/Assignment3/blob/master/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd drive

/content/drive


In [5]:
ls

'My Drive'/  'Shared drives'/


In [6]:
cd My Drive

/content/drive/My Drive


In [7]:
cd NLP\ Embeddings

/content/drive/My Drive/NLP Embeddings


In [8]:
ls

BERT_Fine_Tuning_Sentence_Classification.ipynb        lstm.hdf5
BERT_Fine_Tuning_Sentence_Classification_where.ipynb  Tatti-Copy1.ipynb
bert_test.npy                                         Tatti-Copy3.ipynb
bert_train.npy                                        test.txt
data/                                                 train.txt
embed_copy.npy                                        ytest.npy
embed_test_copy.npy                                   ytrain.npy


In [9]:
!pip install keras-self-attention

In [0]:
class Preprocessing:
    def __init__(self,test,train):
        self.testFile=test
        self.trainFile=train
    def htmlProcess(self,sentence):
        html=-1
        dot=-1
        final=[]
        i=0
        while(i<len(sentence)):
            if(sentence[i].lower().find("http")!=-1):
                html=i
            if((re.search("co",sentence[i].lower()) or re.search("tco",sentence[i].lower())) and html!=-1):
                dot=i
            if(html!=-1 and dot!=-1 and dot<=len(sentence)-2):
                del sentence[html:dot+2]
                i=html
                html=-1
                dot=-1
            i+=1
        return sentence
    def process_file(self):
        for u,i in enumerate([self.testFile,self.trainFile]):
            with open(i) as file:
                data = file.read()
            lines=data.split('\n')
            sentInput=[]
            sentOutput=[]
            dic={'ENG':0,'HIN':1,'O':2}
            mape={}
            i=0
            while(i<len(lines)):
                temp=lines[i].split()
                if(len(temp)>=2):
                    if(temp[0]=='meta' and len(temp)==3):
                        mape[temp[1]]=len(sentInput)
                        if(len(sentInput)!=0):
                            sentInput[-1]=self.htmlProcess(sentInput[-1])
                        sentInput.append([])
                        sentOutput.append(temp[2])
                    else:
                        if(temp[0].find("@")==len(temp[0])-1 or temp[0].find("#")==len(temp[0])-1):
                            i+=1
                        else:
                            data_clean=re.sub("[^a-zA-Z]+", "",temp[0])
                            if(len(data_clean)!=0):
                                sentInput[-1].append(data_clean)
                i+=1
            sentInput[-1]=self.htmlProcess(sentInput[-1])
            if(u==0):
                self.testInput,self.testOutput,self.test_map=sentInput,sentOutput,mape
            else:
                self.trainInput,self.trainOutput,self.train_map=sentInput,sentOutput,mape


In [0]:
obj=Preprocessing("test.txt","train.txt")
obj.process_file()

In [0]:
X_train_raw=obj.trainInput
X_test_raw=obj.testInput

In [0]:
!pip install flair

In [0]:
from flair.data import Sentence

X_testFlair = [Sentence(" ".join(tweet)) for tweet in X_test_raw]
X_trainFlair=[Sentence(" ".join(tweet)) for tweet in X_train_raw]

In [0]:
from flair.embeddings import FlairEmbeddings, BertEmbeddings
# init multilingual BERT
bert_embedding = BertEmbeddings('bert-base-multilingual-cased')

In [0]:
X_trainB = []
for sent in X_trainFlair:
  sentEmb = np.array(bert_embedding.embed(sent)[0])
  emb = []
  for token in sentEmb:
    emb.append(np.array(token.embedding.cpu()))
  X_trainB.append(np.array(emb))
X_testB = []
for sent in X_testFlair:
  sentEmb = np.array(bert_embedding.embed(sent)[0])
  emb = []
  for token in sentEmb:
    emb.append(np.array(token.embedding.cpu()))
  X_testB.append(np.array(emb))

In [0]:
np.save("embed_copy",X_trainB)
np.save("embed_test_copy",X_testB)

In [0]:
import numpy as np
import re
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, Bidirectional,Flatten,GRU
from keras.layers import Embedding
from keras.layers import SimpleRNN
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras import backend as K
from keras_self_attention import SeqSelfAttention
npEmb_train=np.load("bert_train.npy",allow_pickle=1)
npEmb_test=np.load("bert_test.npy",allow_pickle=1)

In [20]:
import torch
from torch.nn.utils.rnn import pad_sequence
print(npEmb_train[0].shape)  
print(npEmb_test.shape)  
X_train = npEmb_train
X_test = npEmb_test
embList = []
for i in X_train:
  embList.append(torch.tensor(i[:31]))
X_train = pad_sequence(embList, batch_first=True)
#X_test = preprocessing.sequence.pad_sequences(X_test, maxlen=500)
embList = []
for i in X_test:
  embList.append(torch.tensor(i[:31]))
X_test = pad_sequence(embList, batch_first=True)

(18, 3072)
(1869,)


In [0]:
l=[]
m=[]
npEmbTrain=np.load("embed_copy.npy",allow_pickle=1)
npEmbTest=np.load("embed_test_copy.npy",allow_pickle=1)

embeddings_polarity=npEmbTrain.tolist()
for i in range(len(embeddings_polarity)):
    l.append(embeddings_polarity[i][0])
    if embeddings_polarity[i][1].lower()=='positive':
        m.append(1)
    elif embeddings_polarity[i][1].lower()=='negative':
        m.append(2)
    else:
        m.append(0)
l1=[]
m1=[]
embeddings_test=npEmbTest.tolist()
for i in range(len(embeddings_test)):
    l1.append(embeddings_test[i][0])
    if embeddings_test[i][1].lower()=='positive':
        m1.append(1)
    elif embeddings_test[i][1].lower()=='negative':
        m1.append(2)
    else:
        m1.append(0)

In [0]:
Y_train=np.load("Ytrain.npy")
Y_test=np.load("ytest.npy")

In [23]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def LSTM_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(128,input_shape=(35,3072),return_sequences=1)))
    model.add(Dropout(0.2))
    model.add(SeqSelfAttention(attention_width=15,attention_activation='sigmoid'))
    model.add(Dense(3, activation='softmax'))
    model.add(GRU(20))
    model.add(Dropout(0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['acc',f1_m,precision_m, recall_m])
    #print(model.summary())
    filepath="lstm.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]
    model.fit(X_train.numpy(), Y_train, epochs=10, batch_size=256,callbacks=callbacks_list)
    return model
model=LSTM_model()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/10





15131/15131 [==============================] - 17s 1ms/step - loss: 1.0087 - acc: 0.4634 - f1_m: 0.1665 - precision_m: 0.5121 - recall_m: 0.1058

Epoch 00001: loss improved from inf to 1.00873, saving model to lstm.hdf5
Epoch 2/10
15131/15131 [==============================] - 11s 753us/step - loss: 0.9085 - acc: 0.5447 - f1_m: 0.3757 - precision_m: 0.6627 - recall_m: 0.2671

Epoch 00002: loss improved from 1.00873 to 0.90845, saving model to lstm.hdf5
Epoch 3/10
15131/15131 [==============================] - 11s 743us/step - loss: 0.8696 - acc: 0.5790 - f1_m: 0.4669 - precision_m: 0.6512 - recall_m: 0.3665

Epoch 00003: loss improved from 0.90845 to 0.86962, saving model to lstm.hdf5
Epoch 4/10
15131/15131 [==============================] - 11s 742us/ste

In [25]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, Y_test, verbose=0)
accuracy, f1_score, precision,recall

(0.5141787053493971, 0.5078455995212205, 0.5159723916240017, 0.50026752293078)

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 31, 256)           3277824   
_________________________________________________________________
dropout_1 (Dropout)          (None, 31, 256)           0         
_________________________________________________________________
seq_self_attention_1 (SeqSel (None, 31, 256)           16449     
_________________________________________________________________
dense_1 (Dense)              (None, 31, 3)             771       
_________________________________________________________________
gru_1 (GRU)                  (None, 20)                1440      
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                

In [0]:
y_pred=model.predict_classes(X_test)

In [33]:
y_dct = {'negative':0, "neutral":1,"positive":2}
y_test_int = []
for i in range(len(Y_test)):
      y_test_int.append(list(Y_test[i]).index(1))
from sklearn.metrics import classification_report as clf
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pandas as pd

target_names=['negative','neutral','positive']

df_cm = pd.DataFrame(confusion_matrix(y_test_int, y_pred), columns=target_names, index=target_names)
df_cm.index.name = 'Actual'
df_cm.columns.name = 'Predicted'
print(df_cm)
print("\n")
print("Accuracy: ",accuracy_score(y_test_int, y_pred))
print("\n","Report:")
print(clf(y_test_int, y_pred, target_names=target_names))

Predicted  negative  neutral  positive
Actual                                
negative        402      110        21
neutral         322      334        98
positive        133      224       225


Accuracy:  0.5141787051899411

 Report:
              precision    recall  f1-score   support

    negative       0.47      0.75      0.58       533
     neutral       0.50      0.44      0.47       754
    positive       0.65      0.39      0.49       582

    accuracy                           0.51      1869
   macro avg       0.54      0.53      0.51      1869
weighted avg       0.54      0.51      0.51      1869

